In [14]:
import argparse
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from io import FileIO
import csv
import pandas as pd

In [2]:
from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

In [3]:
API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'

CLIENT_SECRETS_FILE = 'C:/Users/minha/Desktop/client_secret_558997714597-6tddji99vgbrka974vcnt430sfsdavso.apps.googleusercontent.com.json'

In [4]:
def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename='output.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

In [ ]:
if __name__ == '__main__':
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    youtubeAnalytics = get_service()
    result = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==MINE',
        startDate='2022-01-01',
        endDate='2023-08-05',
        metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost,comments,averageViewDuration',
        dimensions='day',
        sort='day'
    )
    headers = ['date','estMinutesWatched', 'views', 'likes', 'dislikes', 'subscribersGained', 'subscribersLost', 'comments','avgviewdur']
    create_table(result['rows'], headers=headers)
    create_csv(result['rows'], headers=headers)
    
    print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))
    #print(result)

In [ ]:
pd_table = pd.DataFrame(create_table(result['rows'], headers=headers))
pd_table



#pd_table.to_csv('PandasOutput.csv',index = False, header = True, sep = ',', encoding = 'utf-8')



In [ ]:
type(result)